### Part 1. Exploratory Data Analysis.

#### 1.1 Load the data.

In [83]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.dates import MonthLocator, DateFormatter
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
sns.set_style('whitegrid')
pd.set_option('display.float_format', '{:.2f}'.format)

In [84]:
#df = pd.read_csv('groupon.txt', sep=None, engine='python')
#df.to_parquet('groupon.parquet')

In [85]:
df = pd.read_parquet('groupon.parquet')

In [86]:
df.head()

,refund_bucket,refund_sub_bucket,order_date,transaction_date,week_end_date,dmm_subcat_1,category_1,deal_supply_channel,buyer_name_1,auth_bookings,capture_bookings,refunds,cancel_refunds,refunded_units,auth_refunds,capture_units
0,Other,Other,8/4/2016,8/4/2016,8/7/2016,Inverse Normal,Probability distribution II,Goods Stores,Asher,?,?,91.87,?,3,?,?
1,Returns,Change of mind,8/31/2018,9/21/2018,9/23/2018,Binomial Distribution.,Probability distribution I,Goods,Jesus,?,?,20.98,?,1,?,?
2,Fraud,Fraud,4/19/2017,4/19/2017,4/23/2017,Power series,Calculus II,Goods,Tristan,?,?,?,79.94,?,?,?
3,Two-Hour Refunds,Two-Hour Refunds,2/5/2016,2/5/2016,2/7/2016,Prime Factorization Algorithms,?,Goods,Jeremiah,?,?,49.267469958,?,1,?,?
4,Shortage Cancellations,Vendor Shortage,7/21/2018,8/15/2018,8/19/2018,Transformations,Geometry,Goods,Jacob,?,?,29.97,?,2,?,?


Our dataframe consists of 16 columns and approximately 6.8 millions rows. The dataframe is huge as it occupies almost 1 GB of memory. Python loaded all columns as objects - so we need to transform data into right data type. The original dictionary with data types description is provided below.

|Column_name	                |Type		|Description
| --- | --- | --- |
|refund_bucket                 	|Varchar	|Reason for refunding customer
|refund_sub_bucket             	|Varchar	|Sub reason for refunding customer
|transaction_date              	|Date		|date of refund
|week_end_date                 	|Date		|weekend date of refund
|dmm_subcat                    	|Varchar	|sub category of product
|category                      	|Varchar	|category of product
|deal_supply_channel           	|Varchar	|channel of sale
|buyer_name                    	|Varchar	|name of buyer who sourced the product
|auth_bookings                 	|Float		|bookings authorized on card
|capture_bookings              	|Float		|bookings captured
|refunds                       	|Float		|amount of refund
|cancel_refunds                	|Float		|refunds if the transaction was a cancellation
|refunded_units                	|Integer	|quantity of product for which refunds were issued
|auth_refunds                  	|Integer	|	
|capture_units                 	|Integer	|	

To allow a faster processing and easy data manipulation let's subset our dataframe to separate one year of transactions and refunds based on a transaction date. I will take 2018 data. 

To do so I need to convert all date's columns to Python date-time format and will use a boolean mask to create a subset.

In [87]:
df['order_date'] = pd.to_datetime(df['order_date'], infer_datetime_format=True)
df['transaction_date'] = pd.to_datetime(df['transaction_date'], infer_datetime_format=True)
df['week_end_date'] = pd.to_datetime(df['week_end_date'], infer_datetime_format=True)

Since I have already converted date's columns to date format, I can set a desired date to filter a dataframe. Then I will assign a mask to dateaframe.

In [88]:
start_date = '2018-01-01'
end_date = '2018-12-31'
mask = (df['transaction_date'] >= start_date) & (df['transaction_date'] <= end_date)
subset = df.loc[mask]
subset.shape

(2162199, 16)

#### 1.2. Data Cleaning and Transformation

In [89]:
subset = subset.copy()

In [90]:
subset['refunded_units'] = subset['refunded_units'].str.replace('?', '0').astype(int)

Now when we convereted refunded units into integer data type, let's work with auth_refunds, and capture_units in the same way.

In [91]:
subset['auth_refunds'] = subset['auth_refunds'].str.replace('?', '0').astype(int)
subset['capture_units'] = subset['capture_units'].str.replace('?', '0').astype(int)

Now let's convert auth_bookings, capture_bookings, refunds, and cancel_refunds columns into the float data type.

In [92]:
subset['auth_bookings'] = subset['auth_bookings'].str.replace('?', '0').astype(float)
subset['capture_bookings'] = subset['capture_bookings'].str.replace('?', '0').astype(float)
subset['refunds'] = subset['refunds'].str.replace('?', '0').astype(float)
subset['cancel_refunds'] = subset['cancel_refunds'].str.replace('?', '0').astype(float)

I am done with data type transformations. Now let's replace question marks with unknown values in our categorical columns and see what categories and how many of them do I have.

I have 139 unique subcategories in dummy subcategory column.

In [93]:
subset['dmm_subcat_1'] = subset['dmm_subcat_1'].str.replace('?', 'unknown')
print(subset['dmm_subcat_1'].nunique())
# print(subset['dmm_subcat_1'].value_counts().head(10))

138


I have only 14 categories compare to 139 subcategories. In reality categories are actual product categories that Groupon sells online but for our project purposes they were replaced with college math classes names.

In [94]:
subset['category_1'] = subset['category_1'].str.replace('?', 'unknown')
print(subset['category_1'].nunique())

14


There are only two unique supply channels: goods or goods stores.

In [95]:
subset['deal_supply_channel'].value_counts()

Goods           1935944
Goods Stores     226255
Name: deal_supply_channel, dtype: int64

Buyer names are not the actual buyers but rather the employees who sell the particular products. There are 273 unique employees in our subset.

In [96]:
subset['buyer_name_1'] = subset['buyer_name_1'].str.replace('?', 'Unknown')
print(subset['buyer_name_1'].nunique())

280


Our refund bucket column consists of seven buckets, which are general reasons for return.
In addition, we have 18 subbuckets columns that also specify the refund reasons.

In [ ]:
subset['refund_bucket'] = subset['refund_bucket'].str.replace('?', 'Unknown')
subset['refund_sub_bucket'] = subset['refund_sub_bucket'].str.replace('?', 'Unknown')
print(subset['refund_sub_bucket'].nunique())

#### 1.3 Data Analysis and Visualization.

Let's create year, month, and day columns to plot refund and transaction amounts

In [ ]:
variable = pd.DatetimeIndex(subset['transaction_date'])
subset['year'],subset['month'],subset['day'] = variable.year, variable.month, variable.day

In [ ]:
avg_data = subset.groupby(['month','day']).agg({'refunds':np.mean,'capture_bookings':np.mean})
avg_data.index = [datetime(2018,month,day) for (month, day) in avg_data.index]

In [ ]:
plt.figure()
plt.figure(figsize=(18,6))

# subplot 1
plt.subplot(1,2,1)
avg_data['refunds'].plot(colormap='seismic',alpha=0.4)
plt.text('2018-12-25',62,'Christmas',ha='center')
plt.text('2018-01-01',63.5,"New Year's Day",ha='center')
plt.text('2018-02-14',50,"Valentine's Day",ha='center')
plt.ylabel('Average refund amount, $')
plt.title(r'Groupon Average Refund Amount')
plt.margins(0.010, 0.025)

# subplot 2
plt.subplot(1,2,2)
avg_data['capture_bookings'].plot(colormap='autumn', alpha=0.4)
plt.text('2018-11-23',2675,'Thanksgiving',ha='center')
plt.ylabel('Average transaction amount, $')
plt.title(r'Groupon Average Transaction Amount')
plt.margins(0.010, 0.025)
plt.show()

From the following plots, we can clearly see that refunds and transactions have spikes and valleys. A clear seasnal trend can be detected here. We can observe the biggest transaction spike during Thanksgiving, when most people shop for Christmas gifts. We can also observe the highest amount of refunds during Christmas, New Year's, and Valentine's Day, when people return gifts they didn't like.

In [ ]:
for bucket, frame in subset.groupby('refund_bucket'):
    avg = np.round(np.average(frame['refunds']),2)
    print('The average amount of refund in ' + bucket + ' was ' + str(avg))

Groupon might not be able to decrease refunds in Other, Returns, or Two-Hour Refunds because of the nature of these refunds. However, the company can definetly decrease refunds in such categories as Logistic Cancellations, Shortage Cancellations, and Fraud because logistic and shortages are managed by the company.

When we take a closer look at the average amounts of refunds by sub bucket, we see that the same rule applies to sub category column - there are definetly areas for improvement, especially in such sub buckets as Groupon error, Other Shortage, Shipping Issues, 
Warehouse Shortage, etc.

In [ ]:
subset.groupby(['refund_bucket','refund_sub_bucket']).agg(
min_refund=pd.NamedAgg(column='refunds', aggfunc=np.min),
min_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.min),
max_refund=pd.NamedAgg(column='refunds', aggfunc=np.max),
max_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.max),
avg_refund=pd.NamedAgg(column='refunds', aggfunc=np.mean),
avg_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.mean))

Grouping data by refund bucket gives us a very useful insight about the dataset: all valid transactions with no refunds fall into a category 'Unknown' under a refund bucket column, which makes the 'unknown' refund bucket the biggest refund category in the dataset. This fact makes data analysis harder because I don't actually know what products under each particular refund category were bought the most.

As explained by Groupon team, they do not use refund amount to analyze refunds. The better measure for this is a refund rate - refund amount compared to total transaction amount.

Let's add a refund rate column to our dataset to generate some insights.

In [ ]:
by_category = subset.groupby(['category_1']).agg(
min_refund=pd.NamedAgg(column='refunds', aggfunc=np.min),
min_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.min),
max_refund=pd.NamedAgg(column='refunds', aggfunc=np.max),
max_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.max),
avg_refund=pd.NamedAgg(column='refunds', aggfunc=np.mean),
avg_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.mean)).sort_values(by = 'avg_transaction',ascending=False)
by_category['refund_rate'] = (by_category['avg_refund']/by_category['avg_transaction'])*100
by_category

In [ ]:
sns.set_style('white')
by_category.sort_values('refund_rate', inplace = True, ascending=False)
x = by_category.index
y = by_category['refund_rate']
plt.figure(figsize=(16,6))

barlist = plt.bar(x,y,color='grey',edgecolor='k',alpha=0.5)
barlist[0].set_color('lawngreen')
plt.title('Refund Rates by Product Category',fontsize=11)
#plt.tight_layout()
plt.margins(0,0)

# rotate the tick labels for the x axis
_ = plt.gca().xaxis
for item in _.get_ticklabels():
    item.set_rotation(45)
    
# remove all the ticks (both axes), and tick labels on the Y axis
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False,labelbottom=True)

# remove the frame of the chart
for spine in plt.gca().spines.values():
    spine.set_visible(False)
    
# direct label each bar with Y axis values
for bar in barlist:
    plt.gca().text(bar.get_x() + bar.get_width()/2, bar.get_height() - 2, str(int(bar.get_height())) + '%', 
                 ha='center', color='k', fontsize=11)
    
plt.show()

From the following graph, we can see that category Ensemble Methods has the highest refund rate, together with the lowest average amount of transactions and refunds as well. Operations research has second highest refund rate with the third lowest average amount of transactions and refunds as well. If these product categories are not that profitable and have the high refund rate, it is worth to take a closer look and make a decision if these products are worth selling.

In [ ]:
by_subcategory = subset.groupby(['category_1','dmm_subcat_1']).agg(
min_refund=pd.NamedAgg(column='refunds', aggfunc=np.min),
min_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.min),
max_refund=pd.NamedAgg(column='refunds', aggfunc=np.max),
max_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.max),
avg_refund=pd.NamedAgg(column='refunds', aggfunc=np.mean),
avg_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.mean))
by_subcategory['refund_rate'] = (by_subcategory['avg_refund']/by_subcategory['avg_transaction'])*100
by_subcategory.sort_values(by=['avg_transaction','avg_refund','refund_rate'], ascending=False).head(10)

The highest refund rate belongs to buyer Zoey but she is very productive employee with the highest amount of average transaction.

In [ ]:
by_buyer = subset.groupby(['buyer_name_1']).agg(
min_refund=pd.NamedAgg(column='refunds', aggfunc=np.min),
min_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.min),
max_refund=pd.NamedAgg(column='refunds', aggfunc=np.max),
max_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.max),
avg_refund=pd.NamedAgg(column='refunds', aggfunc=np.mean),
avg_transaction=pd.NamedAgg(column='capture_bookings', aggfunc=np.mean))
by_buyer['refund_rate'] = (by_buyer['avg_refund']/by_buyer['avg_transaction'])*100
by_buyer.sort_values(by=['avg_transaction','avg_refund','refund_rate'], ascending=False).head(10)

In [ ]:
# subset.pivot_table(values='refunds', index='buyer_name_1', columns='refund_bucket', aggfunc = np.mean)

### Part 2. Machine Learning Modeling.

In [ ]:
y = subset['refunds']
X = subset.copy()

In [ ]:
columns_to_drop = ['order_date','week_end_date','auth_bookings','refunds','cancel_refunds','refunded_units','auth_refunds',         
'capture_units','year','month','day']

In [ ]:
X.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
X.info()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=0)

I will need to use a customized label encoder as skikit learn label encoder only takes 1-d array as an argument. Please refer to [StackOverflow](https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# creating a customized label encoder as skikit learn label encoder only takes 1-d array as an argument
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
       Transforms columns of X specified in self.columns using
       LabelEncoder(). If no columns specified, transforms all
       columns in X.
       '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
categorical_var = ['refund_bucket','refund_sub_bucket','dmm_subcat_1','category_1','deal_supply_channel',
                   'buyer_name_1']
X_train = MultiColumnLabelEncoder(columns = categorical_var).fit_transform(X_train)
X_train.head()

In [ ]:
X_test = MultiColumnLabelEncoder(columns=categorical_var).transform(X_test)

In [ ]:
X_train.drop(['transaction_date'],inplace=True, axis=1)
X_test.drop(['transaction_date'],inplace=True, axis=1)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True).fit(X_train, y_train)
predictions_reg = reg.predict(X_test)

In [ ]:
reg.score(X_test,y_test)